In [ ]:
import warnings
warnings.filterwarnings('ignore')
from glob import glob 
import sys
sys.path.append('./')
from modules import dendro_misc, dendro_clusters

In [ ]:
# Define names and filenames...

galaxy = 'ngc7496'
galaxy_cat = galaxy
galaxy_hst = galaxy
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies'

hstha_file = '%s/data_hstha/%s/hst_contsub/%s_hst_ha_sic.fits' %(root_dir, galaxy_hst, galaxy_hst)
muscat_file = '%s/data_hstha/%s/muse/%s_nebmask.fits' %(root_dir, galaxy_hst, galaxy.upper())
musha_file = '%s/data_hstha/%s/muse/%s-*_MAPS.fits' %(root_dir, galaxy_hst, galaxy.upper())
musha_file = glob(musha_file)[0] #because of resolution in name

cutout_dir = '%s/data_hstha_nebulae_catalogue/%s/cutouts' %(root_dir, galaxy_hst)
dendro_dir = '%s/data_hstha_nebulae_catalogue/%s/dendro' %(root_dir, galaxy_hst)
cutouts_hdus_dir = '%s/data_hstha_nebulae_catalogue/%s/cutouts_hdus' %(root_dir, galaxy_hst)

rerun_all = True
rerun_masking = True
rerun_getcutouts = True

regions_file = '%s/sample.reg' %cutout_dir
regions_pickel_file = '%s/sample.pickel' %cutout_dir
sample_table_file = '%s/data_misc/sample_table/phangs_sample_table_v1p6.fits' %root_dir
muscat_table_file = '%s/data_misc/nebulae_catalogue/Nebulae_catalogue_v3.fits' %root_dir

cluster_table_file = '%s/data_misc/multiscale_stellar_associations/%s/vselect/ws32pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-vselect-ws32pc-main.fits' %(root_dir, galaxy_cat, galaxy_cat)
cluster_mask_file =  '%s/data_misc/multiscale_stellar_associations/%s/vselect/ws32pc/PHANGS_IR4_hst_wfc3_%s_v1p3_multi_assoc-vselect-ws32pc-idmask.fits' %(root_dir, galaxy_cat, galaxy_cat)

print(hstha_file)
print(muscat_file)
print(musha_file)
print(cutout_dir)
print(dendro_dir)
print(cutouts_hdus_dir)
print(regions_file)
print(regions_pickel_file)
print(sample_table_file)
print(muscat_table_file)
print(cluster_table_file)
print(cluster_mask_file)

In [ ]:
# Load regions, sample table and HDUs... 

hdus_cutouts  = dendro_misc.load_pickle('%s/hdus_all.pickel' %cutout_dir)
regions       = dendro_misc.load_pickle(regions_pickel_file)
sample_table  = dendro_misc.get_galaxyprops(galaxy, sample_table_file)
muscat_table  = dendro_misc.get_museprops(galaxy, muscat_table_file)
hdus          = dendro_misc.load_pickle('%s/hdus_all_withmasked_withdendro.pickel' %dendro_dir)
props_all     = dendro_misc.load_pickle('%s/props_all.pickel' %dendro_dir)

In [ ]:
cluster_table, cluster_mask = dendro_clusters.load_clusters(cluster_table_file, cluster_mask_file)

In [ ]:
if rerun_getcutouts:
    hdus = dendro_clusters.get_clusters_cutouts(cluster_mask, hdus, props_all)
    dendro_misc.save_pickle(hdus, '%s/hdus_all_withmasked_withdendro_withclusters.pickel' %dendro_dir)
else:
    hdus = dendro_misc.load_pickle('%s/hdus_all_withmasked_withdendro_withclusters.pickel' %dendro_dir)

In [ ]:
# Get clusters withing the Dendro leaves, and export region file of these clusters

props_all_cluster = dendro_clusters.get_clusters_all(cluster_table, hdus, props_all)

In [ ]:
props_all_cluster

In [ ]:
# Save table with appended cluster information

# Only take non-masked values in table
props_all_cluster_masked = props_all_cluster[~props_all_cluster.mask['reg_dolflux_Ebv_MinChiSq']]

dendro_misc.save_pickle(props_all_cluster_masked, '%s/props_all_cluster.pickel' %dendro_dir)        
props_all_cluster_masked.write('%s/props_all_cluster.fits' %dendro_dir, overwrite=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(12,3))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)

ax1.hist(props_all_cluster_masked['reg_dolflux_Age_MinChiSq'], alpha=0.3, fc='C0', bins=np.linspace(0,20,10))
ax1.hist(props_all_cluster_masked['reg_dolflux_Age_MinChiSq'], fc='none', ec='C0', lw=1.5, bins=np.linspace(0,20,10))
ax1.set_ylabel('#')
ax1.set_xlabel('Cluster age, log(Myr)')
ax1.grid(alpha=0.3, linestyle=':')
ax1.set_xlim(0,20)

ax2.hist(np.log10(props_all_cluster_masked['reg_dolflux_Mass_MinChiSq']), alpha=0.3, fc='C1')
ax2.hist(np.log10(props_all_cluster_masked['reg_dolflux_Mass_MinChiSq']), fc='none', ec='C1', lw=1.5)
ax2.set_ylabel('#')
ax2.set_xlabel('Cluster mass, log(Msun)')
ax2.grid(alpha=0.3, linestyle=':')

ax3.hist(props_all_cluster_masked['reg_dolflux_Ebv_MinChiSq'], alpha=0.3, fc='C2')
ax3.hist(props_all_cluster_masked['reg_dolflux_Ebv_MinChiSq'], fc='none', ec='C2')
ax3.set_ylabel('#')
ax3.set_xlabel('EBV (mag)')
ax3.grid(alpha=0.3, linestyle=':')

fig.tight_layout()